In [ ]:
import pandas as pd
from pathlib import Path
import seaborn as sns

# Demuxlet report

In [ ]:
demux = []
for f in snakemake.input["demuxlet"]:
    r = Path(f).parent.name
    df = pd.read_csv(f, sep="\t")
    df["Channel"] = r
    df = df[
        [
            "BARCODE",
            "DROPLET.TYPE",
            "SNG.BEST.GUESS",
            "DBL.BEST.GUESS",
            "BEST.LLK",
            "Channel",
        ]
    ]
    df.set_index("BARCODE", inplace=True)
    df.rename(
        columns={
            "DROPLET.TYPE": "Droplet Type",
            "SNG.BEST.GUESS": "Singlet Best Guess",
            "DBL.BEST.GUESS": "Doublet Best Guess",
            "BEST.LLK": "Best Log Likelihood",
        },
        inplace=True,
    )
    demux.append(df)
demux = pd.concat(demux)

In [ ]:
fig = sns.countplot(demux, x="Channel", hue="Droplet Type")
sns.move_legend(fig, "upper left", bbox_to_anchor=(1, 1))
fig.set(xlabel="Channel", ylabel="# Cells")

In [ ]:
cm = sns.color_palette("flare", as_cmap=True)
singlets = (
    demux.loc[demux["Droplet Type"] == "SNG"]
    .groupby(["Channel", "Singlet Best Guess"])
    .count()["Droplet Type"]
    .unstack(fill_value=0)
)
singlets.style.background_gradient(cmap=cm)